In [4]:
import os
os.environ['http_proxy'] = "http://127.0.0.1:3128" 
os.environ['https_proxy'] = "http://127.0.0.1:3128"

In [5]:
from portal_analytics.api import AnalyticsApi
import portal_analytics.parser as parser
import portal_analytics.create_rasters as cr
from shapely.geometry import box, Point
import json
import pandas as pd

In [6]:
with open('items.json') as file:
      items = json.load(file)
layers=  items["layers"]
for layer in layers:
    print(layer)

mineral_occurrences
mines
mineral_tenements
scanned_250K_maps
nvcl
all_boreholes


In [7]:
KEY_FILE = 'GeosciencePortal-eeac25f8efb3.json'
ACCOUNT_NAME = 'AUSGIN website'
PROPERTY_NAME = 'AUSGIN'
PROFILE_NAME = 'AUSGIN - exclude internal traffic'


In [8]:
connection=AnalyticsApi(KEY_FILE,ACCOUNT_NAME,PROPERTY_NAME,PROFILE_NAME)

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:
results = connection.get_results()

In [ ]:
raw_results = parser.create_dataframe(results)
raw_results.to_excel("raw_results.xlsx")

In [ ]:
data_frame = parser.process_dataframe(raw_results)
data_frame["ga:totalEvents"]=data_frame["ga:totalEvents"].apply(int)
add_layers=data_frame[data_frame["ga:eventCategory"]=="Add:KnownLayer"].groupby(by="ga:eventAction")["ga:totalEvents"].sum()

In [ ]:
pd.DataFrame(add_layers).sort_values("ga:totalEvents", ascending=False).to_excel("add_layers.xlsx")

In [ ]:
query=raw_results[raw_results["ga:eventCategory"]=="Query"]

In [ ]:
query["ga:eventAction"].unique();

In [ ]:
gdf = parser.create_geodataframe(data_frame)
%matplotlib inline

In [ ]:
map_centre=Point(133.3,-26)

australia_bounds = box(110,-45,155,-10)
buff=map_centre.buffer(1)


In [ ]:
#gdf["distance"] = gdf["shape"].centroid.distance(map_centre)
gdf["area"] = gdf["shape"].area
gdf["weighting"] = australia_bounds.area/ gdf["shape"].area
gdf = gdf[gdf["shape"].centroid.intersects(australia_bounds)]
#gdf = gdf[gdf["shape"].centroid.intersects(buff)]

gdf = gdf[gdf["area"] <= australia_bounds.area]
#gdf = gdf[gdf["area"] > 0]
#gdf


x=gdf[gdf['ga:eventAction'] == "Layer:Scanned 250K Geological Map Index"]


In [ ]:
cr.create_spatial_data(gdf)


In [ ]:
mines=gdf[gdf['ga:eventAction'] == "Layer:Mines"]

In [ ]:
row=mines.iloc(1)[0]

In [ ]:
shape=row["shape"]

In [ ]:
weighting = row["weighting"]

In [ ]:
(x0,y0,x1,y1) = shape.bounds

In [ ]:
pixel_size=0.1


In [ ]:
width = int((x1-x0)/pixel_size)
height = int((y1-y0)/pixel_size)

In [ ]:
grid = cr.get_envelope(width, height)

In [ ]:

grid*weighting

In [ ]:
weighting
from osgeo import gdal, osr

In [ ]:
driver = gdal.GetDriverByName('GTiff')
outRaster = driver.Create("test.tif", width, height, 1, gdal.GDT_Float64) 

In [ ]:
import os


In [ ]:
path=os.path.join("output","mines_01.tif")

In [ ]:
outRaster = driver.Create(path, width, height, 1, gdal.GDT_Float64)

In [ ]:
outRasterSRS=osr.SpatialReference()
outRasterSRS.ImportFromEPSG(4326) # import the EPSG projection
outRaster.SetProjection(outRasterSRS.ExportToWkt()) 
outRaster.SetGeoTransform((x0, pixel_size, 0, y0, 0, pixel_size))
outRaster.GetRasterBand(1).WriteArray(grid)
outRaster.FlushCache()

In [ ]:
import numpy as np
np.ones((20, 20))* weighting

In [ ]:
x0,y0,x1,y1=(151.19, -32.61, 151.2, -32.59)

In [ ]:
width = int((x1-x0)/0.1)
height = int((y1-y0)/0.1)

In [ ]:
width
import math

In [ ]:
math.ceil((x1-x0)/0.1)